In [18]:
import pandas as pd
import numpy as np
import importlib
import warnings
import gc
import sys
sys.path.append("./")
import model_xgboost
import feature_engineering
importlib.reload(model_xgboost)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("../../data/preprocessed/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
df.shape

(31362, 19)

In [4]:
neural_prophet_fe = pd.read_csv("../../data/preprocessed/features_neuralprophet_completo.csv", sep=',', encoding='utf-8')
neural_prophet_fe['ds'] = pd.to_datetime(neural_prophet_fe['ds'], errors='coerce')
# Versión alternativa más robusta:
neural_prophet_fe['periodo'] = neural_prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
neural_prophet_fe = neural_prophet_fe[['periodo', 'product_id', 'trend', "season_yearly", "season_monthly"]]
df = df.merge(neural_prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 22)

In [5]:
prophet_fe = pd.read_csv("../../data/preprocessed/prophet_features_tn_zscore.csv", sep=',', encoding='utf-8')
prophet_fe['ds'] = pd.to_datetime(prophet_fe['ds'], errors='coerce')
prophet_fe['periodo'] = prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
prophet_fe = prophet_fe[['periodo', 'product_id', 'trend_add', "yearly_add", "additive_terms", 'trend_mult', 'yearly_mult', 'multiplicative_terms']]
df = df.merge(prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 28)

In [6]:
df = feature_engineering.get_lags(df, "tn", 201912)
df = feature_engineering.get_delta_lags(df, "tn", 24)
df = feature_engineering.get_rolling_means(df, "tn", 201912)
df = feature_engineering.get_rolling_stds(df, "tn", 201912)
df = feature_engineering.get_rolling_mins(df, "tn", 201912)
df = feature_engineering.get_rolling_maxs(df, "tn", 201912)
df = feature_engineering.get_rolling_medians(df, "tn", 201912)
df = feature_engineering.get_rolling_skewness(df, "tn", 201912)
df = feature_engineering.get_autocorrelaciones(df, "tn", 201912)
df.shape

(31362, 590)

In [7]:
df = feature_engineering.generar_ids(df)
df = feature_engineering.get_componentesTemporales(df)
df = feature_engineering.get_anomaliasPoliticas(df)
# df = feature_engineering.descomposicion_serie_temporal(df, col='tn')
df.shape

(31362, 615)

In [8]:
df = feature_engineering.chatGPT_features_serie(df, "tn")
df = feature_engineering.mes_con_feriado(df)
df.shape

(31362, 644)

In [9]:
df = feature_engineering.get_dolar(df)
df = feature_engineering.get_IPC(df)
df['ipc'] = df['ipc'].str.replace(',', '.').astype(float)
df['dolar'] = df['dolar'].str.replace(',', '.').astype(float)
# df.drop(columns=['ds'], inplace=True)
df.fillna(0, inplace=True) ##### EXPERIMENTAR
df = feature_engineering.correlacion_exogenas(df)
df = feature_engineering.get_mes_receso_escolar(df)
df.shape

(31362, 647)

In [10]:
df = feature_engineering.create_ratio_features(df)
df = feature_engineering.enhance_lifecycle_features(df)
df = feature_engineering.create_category_features(df)
df = feature_engineering.create_regime_features(df)
df = feature_engineering.create_nonlinear_trends(df)
df = feature_engineering.create_temporal_interactions(df)
df = feature_engineering.create_asymmetric_window_features(df)
df = feature_engineering.recomendaciones_deepseek(df)
df = feature_engineering.get_nuevas_features(df)
df.shape

(31362, 689)

In [12]:
df = df.select_dtypes(exclude=['datetime', 'datetime64[ns]', 'object', 'category'])
df['target'].fillna(0, inplace=True)
# Reemplazar inf y -inf por NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# (opcional) Llenar NaNs con 0 o eliminar filas
df.fillna(0, inplace=True)  # o df.dropna(inplace=True)


train = df[df['periodo'] <= 201912]
test = df[df['periodo'] == 201912]

model_xgboost.optimizar_xgboost_con_semillerio(train, version="v1")

[I 2025-07-02 08:28:23,794] A new study created in memory with name: no-name-bbe28a84-16ef-4601-8f08-6bba3ac6a981
[I 2025-07-02 08:45:52,555] Trial 0 finished with value: 22226.240698634043 and parameters: {'max_depth': 6, 'learning_rate': 0.20141221954470895, 'min_child_weight': 2, 'subsample': 0.8278800040469727, 'colsample_bytree': 0.5359411502200573, 'reg_alpha': 0.14232887966834085, 'reg_lambda': 0.9517062809779588, 'n_estimators': 596}. Best is trial 0 with value: 22226.240698634043.


Mejor trial hasta ahora: RMSE=22226.2407, Parámetros={'max_depth': 6, 'learning_rate': 0.20141221954470895, 'min_child_weight': 2, 'subsample': 0.8278800040469727, 'colsample_bytree': 0.5359411502200573, 'reg_alpha': 0.14232887966834085, 'reg_lambda': 0.9517062809779588, 'n_estimators': 596}


[I 2025-07-02 09:01:07,953] Trial 1 finished with value: 21601.047649576118 and parameters: {'max_depth': 7, 'learning_rate': 0.13590035365359482, 'min_child_weight': 6, 'subsample': 0.5596925453636922, 'colsample_bytree': 0.6230791089454267, 'reg_alpha': 0.7440590797606599, 'reg_lambda': 0.31856032985665184, 'n_estimators': 655}. Best is trial 1 with value: 21601.047649576118.


Mejor trial hasta ahora: RMSE=21601.0476, Parámetros={'max_depth': 7, 'learning_rate': 0.13590035365359482, 'min_child_weight': 6, 'subsample': 0.5596925453636922, 'colsample_bytree': 0.6230791089454267, 'reg_alpha': 0.7440590797606599, 'reg_lambda': 0.31856032985665184, 'n_estimators': 655}


[I 2025-07-02 09:22:43,428] Trial 2 finished with value: 22585.710880149883 and parameters: {'max_depth': 10, 'learning_rate': 0.16881755827875575, 'min_child_weight': 5, 'subsample': 0.5370562372078382, 'colsample_bytree': 0.5904876430751341, 'reg_alpha': 0.47501363281769204, 'reg_lambda': 0.06825913153569718, 'n_estimators': 671}. Best is trial 1 with value: 21601.047649576118.


Mejor trial hasta ahora: RMSE=21601.0476, Parámetros={'max_depth': 7, 'learning_rate': 0.13590035365359482, 'min_child_weight': 6, 'subsample': 0.5596925453636922, 'colsample_bytree': 0.6230791089454267, 'reg_alpha': 0.7440590797606599, 'reg_lambda': 0.31856032985665184, 'n_estimators': 655}


[I 2025-07-02 09:34:54,086] Trial 3 finished with value: 20200.979667207874 and parameters: {'max_depth': 19, 'learning_rate': 0.07500697651503813, 'min_child_weight': 8, 'subsample': 0.7877308746208516, 'colsample_bytree': 0.6684141281467, 'reg_alpha': 0.6550576297691915, 'reg_lambda': 0.6516343798204542, 'n_estimators': 372}. Best is trial 3 with value: 20200.979667207874.


Mejor trial hasta ahora: RMSE=20200.9797, Parámetros={'max_depth': 19, 'learning_rate': 0.07500697651503813, 'min_child_weight': 8, 'subsample': 0.7877308746208516, 'colsample_bytree': 0.6684141281467, 'reg_alpha': 0.6550576297691915, 'reg_lambda': 0.6516343798204542, 'n_estimators': 372}

Mejores hiperparámetros: {'max_depth': 19, 'learning_rate': 0.07500697651503813, 'min_child_weight': 8, 'subsample': 0.7877308746208516, 'colsample_bytree': 0.6684141281467, 'reg_alpha': 0.6550576297691915, 'reg_lambda': 0.6516343798204542, 'n_estimators': 372}


({'max_depth': 19,
  'learning_rate': 0.07500697651503813,
  'min_child_weight': 8,
  'subsample': 0.7877308746208516,
  'colsample_bytree': 0.6684141281467,
  'reg_alpha': 0.6550576297691915,
  'reg_lambda': 0.6516343798204542,
  'n_estimators': 372,
  'objective': 'reg:squarederror',
  'eval_metric': 'rmse',
  'tree_method': 'hist',
  'verbosity': 0,
  'max_bin': 1024},
 <optuna.study.study.Study at 0x188cb27c7a0>)

In [51]:
result = model_xgboost.semillerio_en_prediccion_xgboost(train, test, version="v1")

In [57]:
result

,periodo,product_id,target,pred
30476,201912,20001,0.0,1571.505737
30477,201912,20002,0.0,1009.046021
30478,201912,20003,0.0,690.817261
30479,201912,20004,0.0,543.897766
30480,201912,20005,0.0,696.299438
...,...,...,...,...
31357,201912,21265,0.0,31.830990
31358,201912,21266,0.0,23.470936
31359,201912,21267,0.0,24.481310
31360,201912,21271,0.0,23.151690


In [56]:
ok = pd.read_csv("../../data/raw/product_id_apredecir201912.csv", sep=',')
ok

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005
...,...
775,21263
776,21265
777,21266
778,21267


In [58]:
kaggle = result[result['product_id'].isin(ok['product_id'].unique())]
kaggle = kaggle[['product_id', 'pred']].rename({'pred':'tn'})

In [64]:
kaggle.to_csv("./xgboost_baseline.csv", sep=',', index=False)

In [15]:
model_xgboost.optimizar_con_optuna_xgboost_db(train, version="v2")

[I 2025-07-02 09:54:02,524] Using an existing study with name 'xgboost_optimization_v2' instead of creating a new one.



Para visualizar los resultados en tiempo real:
1. Abre otra terminal y ejecuta:
   optuna-dashboard sqlite:///optuna_studies_v2.db
2. Abre en tu navegador: http://127.0.0.1:8080/
[09:54:02] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (31356, 683, 21416148).
[09:54:03] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (1, 683, 683).
[09:55:40] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (31357, 683, 21416831).
[09:55:40] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (1, 683, 683).
[09:57:19] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (31358, 683, 21417514).
[09:57:19] INFO: C:\actions-runne

[I 2025-07-02 10:01:49,042] Trial 3 finished with value: 2.757596325874329 and parameters: {'learning_rate': 0.09971090774652261, 'n_estimators': 851, 'max_depth': 8, 'min_child_weight': 5, 'subsample': 0.5148806386299757, 'colsample_bytree': 0.7758305653550759, 'lambda': 3.4247805503994705e-07, 'alpha': 1.3265468281427542e-08, 'max_bin': 278}. Best is trial 1 with value: 1.497717136144638.


Mejor trial hasta ahora: RMSE=1.497717, Parámetros={'learning_rate': 0.06184199691031956, 'n_estimators': 175, 'max_depth': 8, 'min_child_weight': 5, 'subsample': 0.7175142747939403, 'colsample_bytree': 0.8551225199271686, 'lambda': 2.5226202361453683e-05, 'alpha': 0.00035316863789536186, 'max_bin': 112}
[10:01:49] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (31356, 683, 21416148).
[10:01:49] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (1, 683, 683).
[10:03:54] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (31357, 683, 21416831).
[10:03:54] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (1, 683, 683).
[10:05:59] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\it

[I 2025-07-02 10:12:07,238] Trial 4 finished with value: 0.9480952084064483 and parameters: {'learning_rate': 0.017822890445023876, 'n_estimators': 921, 'max_depth': 9, 'min_child_weight': 9, 'subsample': 0.9905789681182895, 'colsample_bytree': 0.5164639526610038, 'lambda': 5.958950389020772e-08, 'alpha': 0.013196646205564539, 'max_bin': 269}. Best is trial 4 with value: 0.9480952084064483.


...

[I 2025-07-02 12:50:49,762] Trial 26 finished with value: 0.45443336516618726 and parameters: {'learning_rate': 0.012005790350734319, 'n_estimators': 990, 'max_depth': 10, 'min_child_weight': 6, 'subsample': 0.9157026317065159, 'colsample_bytree': 0.6845670264156642, 'lambda': 0.008486779317100715, 'alpha': 0.00030601642237244376, 'max_bin': 220}. Best is trial 12 with value: 0.4499828428030014.


Mejor trial hasta ahora: RMSE=0.449983, Parámetros={'learning_rate': 0.027600714294169222, 'n_estimators': 824, 'max_depth': 10, 'min_child_weight': 2, 'subsample': 0.8539770403381404, 'colsample_bytree': 0.990030628129953, 'lambda': 6.077890680976744, 'alpha': 5.8922044120143475e-05, 'max_bin': 176}
[12:50:50] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (31356, 683, 21416148).
[12:50:50] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (1, 683, 683).
[12:54:15] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (31357, 683, 21416831).
[12:54:15] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (1, 683, 683).
[12:57:30] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterat

[I 2025-07-02 13:07:30,775] Trial 27 finished with value: 0.7517881687264889 and parameters: {'learning_rate': 0.013238426253306377, 'n_estimators': 993, 'max_depth': 10, 'min_child_weight': 6, 'subsample': 0.9224051475672816, 'colsample_bytree': 0.6914843840921329, 'lambda': 0.0063463129383736815, 'alpha': 0.00022431169613964933, 'max_bin': 225}. Best is trial 12 with value: 0.4499828428030014.


Mejor trial hasta ahora: RMSE=0.449983, Parámetros={'learning_rate': 0.027600714294169222, 'n_estimators': 824, 'max_depth': 10, 'min_child_weight': 2, 'subsample': 0.8539770403381404, 'colsample_bytree': 0.990030628129953, 'lambda': 6.077890680976744, 'alpha': 5.8922044120143475e-05, 'max_bin': 176}

Mejores hiperparámetros encontrados:
learning_rate: 0.027600714294169222
n_estimators: 824
max_depth: 10
min_child_weight: 2
subsample: 0.8539770403381404
colsample_bytree: 0.990030628129953
lambda: 6.077890680976744
alpha: 5.8922044120143475e-05
max_bin: 176


(<optuna.study.study.Study at 0x188c95df020>,
 {'learning_rate': 0.027600714294169222,
  'n_estimators': 824,
  'max_depth': 10,
  'min_child_weight': 2,
  'subsample': 0.8539770403381404,
  'colsample_bytree': 0.990030628129953,
  'lambda': 6.077890680976744,
  'alpha': 5.8922044120143475e-05,
  'max_bin': 176,
  'objective': 'reg:squarederror',
  'eval_metric': 'rmse',
  'booster': 'gbtree',
  'tree_method': 'hist',
  'verbosity': 0})

In [33]:
result_df = model_xgboost.semillerio_en_prediccion_con_pesos_xgb(train, test, version="v2")

[0]	validation_0-rmse:115.75541
[1]	validation_0-rmse:112.83560
[2]	validation_0-rmse:109.99436
[3]	validation_0-rmse:107.22758
[4]	validation_0-rmse:104.54017
[5]	validation_0-rmse:101.92343
[6]	validation_0-rmse:99.39761
[7]	validation_0-rmse:96.95115
[8]	validation_0-rmse:94.56689
[9]	validation_0-rmse:92.24802
[10]	validation_0-rmse:90.00131
[11]	validation_0-rmse:87.80709
[12]	validation_0-rmse:85.70179
[13]	validation_0-rmse:83.64012
[14]	validation_0-rmse:81.61575
[15]	validation_0-rmse:79.70322
[16]	validation_0-rmse:77.80649
[17]	validation_0-rmse:75.95268
[18]	validation_0-rmse:74.17124
[19]	validation_0-rmse:72.45825
[20]	validation_0-rmse:70.77045
[21]	validation_0-rmse:69.13764
[22]	validation_0-rmse:67.55872
[23]	validation_0-rmse:66.01079
[24]	validation_0-rmse:64.54072
[25]	validation_0-rmse:63.09491
[26]	validation_0-rmse:61.70329
[27]	validation_0-rmse:60.33554
[28]	validation_0-rmse:59.03524
[29]	validation_0-rmse:57.75699
[30]	validation_0-rmse:56.52494
[31]	validat

In [34]:
def get_promedio_780():    
    df = pd.read_csv("../../data/preprocessed/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
    df = df[df['periodo'] >= 201901]  # Filtrar desde 201901

    productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")

    df = df.merge(productos_ok, on='product_id', how='inner')

    df = df.groupby('product_id').agg({'tn': 'mean'}).reset_index()

    return df
promedios = get_promedio_780()
promedios

,product_id,tn
0,20001,1454.732720
1,20002,1175.437142
2,20003,784.976407
3,20004,627.215328
4,20005,668.270104
...,...,...
775,21263,0.029993
776,21265,0.089541
777,21266,0.094659
778,21267,0.092835


In [35]:
result_df = result_df.merge(promedios, how='left', on='product_id')
result_df

,periodo,product_id,target,pred,tn
0,201912,20001,0.0,1307.493408,1454.732720
1,201912,20002,0.0,901.071411,1175.437142
2,201912,20003,0.0,604.131042,784.976407
3,201912,20004,0.0,516.870972,627.215328
4,201912,20005,0.0,487.723450,668.270104
...,...,...,...,...,...
881,201912,21265,0.0,0.278887,0.089541
882,201912,21266,0.0,-0.436438,0.094659
883,201912,21267,0.0,-1.323009,0.092835
884,201912,21271,0.0,-0.219403,NaN


In [36]:
result_df.loc[result_df['pred'] < 0 , 'pred'] = result_df['tn']
result_df

,periodo,product_id,target,pred,tn
0,201912,20001,0.0,1307.493408,1454.732720
1,201912,20002,0.0,901.071411,1175.437142
2,201912,20003,0.0,604.131042,784.976407
3,201912,20004,0.0,516.870972,627.215328
4,201912,20005,0.0,487.723450,668.270104
...,...,...,...,...,...
881,201912,21265,0.0,0.278887,0.089541
882,201912,21266,0.0,0.094659,0.094659
883,201912,21267,0.0,0.092835,0.092835
884,201912,21271,0.0,NaN,NaN


In [37]:
result_df.dropna(inplace=True)
result_df

,periodo,product_id,target,pred,tn
0,201912,20001,0.0,1307.493408,1454.732720
1,201912,20002,0.0,901.071411,1175.437142
2,201912,20003,0.0,604.131042,784.976407
3,201912,20004,0.0,516.870972,627.215328
4,201912,20005,0.0,487.723450,668.270104
...,...,...,...,...,...
879,201912,21263,0.0,0.029993,0.029993
881,201912,21265,0.0,0.278887,0.089541
882,201912,21266,0.0,0.094659,0.094659
883,201912,21267,0.0,0.092835,0.092835


In [38]:
result_df.drop(columns=['tn'], inplace=True)
result_df.rename(columns={'pred':'tn'}, inplace=True)
result_df[['product_id','tn']].to_csv("./predicciones_xgboost_laburo.csv", sep=',', index=False)